In [1]:
! wget -P ../../data http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2020-11-07 15:41:56--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘../../data/ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  1.15MB/s    in 0.8s    

2020-11-07 15:41:57 (1.15 MB/s) - ‘../../data/ml-latest-small.zip’ saved [978202/978202]



In [255]:
import zipfile

with zipfile.ZipFile('../../data/ml-latest-small.zip', 'r') as zip_ref: zip_ref.extractall('../../data')

In [256]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [257]:
links = pd.read_csv('../../data/ml-latest-small/links.csv')

In [258]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [259]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [260]:
movies = pd.read_csv('../../data/ml-latest-small/movies.csv')

In [261]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [262]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [263]:
ratings = pd.read_csv('../../data/ml-latest-small/ratings.csv')

In [264]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [265]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [266]:
tags = pd.read_csv('../../data/ml-latest-small/tags.csv')

In [267]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [268]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [269]:
# # import necessary libraries
# from pyspark.sql import SparkSession
# # instantiate SparkSession object
# # spark = SparkSession.builder.master('local').getOrCreate()
# spark = SparkSession\
#         .builder\
#         .appName('ALSExample').config('spark.driver.host', 'localhost')\
#         .getOrCreate()
# # read in the dataset into pyspark DataFrame
# links = spark.read.csv('../../data/ml-latest-small/links.csv', header='true', inferSchema='true')
# movies = spark.read.csv('../../data/ml-latest-small/movies.csv', header='true', inferSchema='true')
# ratings = spark.read.csv('../../data/ml-latest-small/ratings.csv', header='true', inferSchema='true')
# tags = spark.read.csv('../../data/ml-latest-small/tags.csv', header='true', inferSchema='true')

In [270]:
# merged links and movies
movie_ratings = links.merge(movies, on=["movieId"])
movie_ratings.head()

,movieId,imdbId,tmdbId,title,genres
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,Father of the Bride Part II (1995),Comedy


In [271]:
movie_ratings = ratings.merge(movie_ratings, on=["movieId"])
movie_ratings.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId,title,genres
0,1,1,4.0,964982703,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [272]:
movie_ratings = tags.merge(movie_ratings, on=["movieId"])
movie_ratings.head()

,userId_x,movieId,tag,timestamp_x,userId_y,rating,timestamp_y,imdbId,tmdbId,title,genres
0,2,60756,funny,1445714994,2,5.0,1445714980,838283,12133.0,Step Brothers (2008),Comedy
1,2,60756,funny,1445714994,18,3.0,1455749449,838283,12133.0,Step Brothers (2008),Comedy
2,2,60756,funny,1445714994,62,3.5,1528934376,838283,12133.0,Step Brothers (2008),Comedy
3,2,60756,funny,1445714994,68,2.5,1269123243,838283,12133.0,Step Brothers (2008),Comedy
4,2,60756,funny,1445714994,73,4.5,1464196221,838283,12133.0,Step Brothers (2008),Comedy


In [273]:
movie_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233213 entries, 0 to 233212
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userId_x     233213 non-null  int64  
 1   movieId      233213 non-null  int64  
 2   tag          233213 non-null  object 
 3   timestamp_x  233213 non-null  int64  
 4   userId_y     233213 non-null  int64  
 5   rating       233213 non-null  float64
 6   timestamp_y  233213 non-null  int64  
 7   imdbId       233213 non-null  int64  
 8   tmdbId       233213 non-null  float64
 9   title        233213 non-null  object 
 10  genres       233213 non-null  object 
dtypes: float64(2), int64(6), object(3)
memory usage: 21.4+ MB


# Using Matrix Factorization with ALS to Build a Collaborative Filtering Movie Recommendation Model 

In [274]:
# import PySpark, NumPy, IPython

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf, col, when
from pyspark.ml.evaluation import RegressionEvaluator

import numpy as np

from IPython.display import Image
from IPython.display import display

In [275]:
# generate a spark session
spark = SparkSession.builder.appName('FSM').getOrCreate()

In [276]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [277]:
# create spark dataframes

In [278]:
ratings_df = spark.read.csv('../../data/ml-latest-small/ratings.csv', inferSchema=True, header=True)
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [279]:
ratings_df.show(1)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
+------+-------+------+---------+
only showing top 1 row



In [280]:
movies_df = spark.read.csv('../../data/ml-latest-small/movies.csv', inferSchema=True, header=True)
movies_df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [281]:
movies_df.show(1)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
+-------+----------------+--------------------+
only showing top 1 row



In [282]:
links_df = spark.read.csv('../../data/ml-latest-small/links.csv', inferSchema=True, header=True)
links_df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- imdbId: integer (nullable = true)
 |-- tmdbId: integer (nullable = true)



In [283]:
tags_df = spark.read.csv('../../data/ml-latest-small/tags.csv', inferSchema=True, header=True)
tags_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: integer (nullable = true)



In [284]:
# Split training and testing data
# ALS requires numeric data, our data is already numeric
training_df, validation_df = ratings_df.randomSplit([.8, .2])

In [285]:
# ALS requires the following parameters
iterations = 10
regularization_parameter = 0.1
rank = 4
errors = []
err = 0

In [286]:
# Build the model
als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=4, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training_df)
predictions = model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)
print("Root-mean_square error = " + str(rmse))

Root-mean_square error = 0.8868249823200508


In [124]:
# this code checks rmse for ranks 4 - 10
for rank in range(4,10):
    als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=rank, userCol="userId", itemCol="movieId", ratingCol="rating")
    model = als.fit(training_df)
    predictions = model.transform(validation_df)
    new_predictions = predictions.filter(col('prediction') != np.nan)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    rmse = evaluator.evaluate(new_predictions)
    print("Root-mean_square error = " + str(rmse))

Root-mean_square error = 0.8840305708681246
Root-mean_square error = 0.8877305104104398
Root-mean_square error = 0.8837197252045196
Root-mean_square error = 0.8916516039700236
Root-mean_square error = 0.894832573435923
Root-mean_square error = 0.8926116127840896


In [287]:
# change rank to 8, based on above code, this is the highest rmse score
als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=8, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training_df)
predictions = model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)
print("Root-mean_square error = " + str(rmse))

Root-mean_square error = 0.8772808109883926


In [ ]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.maxIter, [5, 50, 100, 200]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

In [129]:
# this code is supposed to  check cross val scores for ranks 4-10 but takes a long time to run and I couldn't
# figure out how to print out the scores

# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# ls = ALS(maxIter=iterations, regParam=regularization_parameter, rank=rank, userCol="userId", itemCol="movieId", ratingCol="rating")
# paramGrid = ParamGridBuilder() \
#     .addGrid(als.regParam, [0.1, 0.01, 0.18]) \
#     .addGrid(als.rank, range(4, 10)) \
#     .build()
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
# crossval = CrossValidator(estimator=als,
#                           estimatorParamMaps=paramGrid,
#                           evaluator=evaluator,
#                           numFolds=5)
# cvModel = crossval.fit(training_df)

In [288]:
predictions.show(n=5)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|   191|    148|   5.0| 829760897|       NaN|
|   520|    471|   5.0|1326609921| 3.7186322|
|   448|    471|   4.0|1178980875| 3.6630256|
|   608|    471|   1.5|1117161794| 2.5045674|
|   191|    496|   5.0| 829760898|       NaN|
+------+-------+------+----------+----------+
only showing top 5 rows



In [289]:
predictions.join(movies_df, "movieId").select("userId","title","genres","prediction").show(5)

+------+--------------------+--------------------+----------+
|userId|               title|              genres|prediction|
+------+--------------------+--------------------+----------+
|   191|Awfully Big Adven...|               Drama|       NaN|
|   520|Hudsucker Proxy, ...|              Comedy| 3.7186322|
|   448|Hudsucker Proxy, ...|              Comedy| 3.6630256|
|   608|Hudsucker Proxy, ...|              Comedy| 2.5045674|
|   191|What Happened Was...|Comedy|Drama|Roma...|       NaN|
+------+--------------------+--------------------+----------+
only showing top 5 rows



In [290]:
for_one_user = predictions.filter(col("userId")==599).join(movies_df, "movieId").join(links_df, "movieId").select("userId","title","genres","tmdbId","prediction")
for_one_user.show(5)

+------+--------------------+--------------------+------+----------+
|userId|               title|              genres|tmdbId|prediction|
+------+--------------------+--------------------+------+----------+
|   599|American Splendor...|        Comedy|Drama|  2771| 3.4156573|
|   599|     Sky High (2003)|Action|Horror|Thr...|  5846|       NaN|
|   599|Bonnie and Clyde ...|         Crime|Drama|   475| 3.2393277|
|   599|      Titanic (1997)|       Drama|Romance|   597|  2.492839|
|   599|Prestige, The (2006)|Drama|Mystery|Sci...|  1124| 3.2368648|
+------+--------------------+--------------------+------+----------+
only showing top 5 rows



In [291]:
# create code to open web browsers for recommended movies
import webbrowser
link = "https://www.themoviedb.org/movie/"
for movie in for_one_user.take(1):
    movieURL = link+str(movie.tmdbId)
    print(movie.title)
    webbrowser.open(movieURL)

American Splendor (2003)


In [292]:
# generate top 5 movie recommendations for each user
userRecomments = model.recommendForAllUsers(5)
# generate top 5 user recommendations for each movie
movieRecomments = model.recommendForAllItems(5)

In [293]:
userRecomments.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [294]:
userRecomments.select("userId","recommendations.movieId").show(5, False)

+------+-----------------------------------+
|userId|movieId                            |
+------+-----------------------------------+
|471   |[6818, 89904, 6460, 7096, 213]     |
|463   |[7842, 59018, 7841, 60943, 6818]   |
|496   |[6818, 25771, 148881, 96004, 40491]|
|148   |[89904, 86320, 98491, 2843, 49347] |
|540   |[7842, 96004, 60943, 59018, 7841]  |
+------+-----------------------------------+
only showing top 5 rows



In [295]:
movieRecomments.select("movieId","recommendations.UserId").show(5, False)

+-------+------------------------+
|movieId|UserId                  |
+-------+------------------------+
|1580   |[53, 12, 543, 452, 43]  |
|4900   |[53, 371, 452, 122, 523]|
|5300   |[375, 53, 296, 535, 295]|
|6620   |[236, 418, 375, 535, 53]|
|7340   |[43, 594, 578, 53, 162] |
+-------+------------------------+
only showing top 5 rows



In [296]:
users = ratings_df.select("userId").distinct().limit(3);
users.show()

+------+
|userId|
+------+
|   148|
|   463|
|   471|
+------+



In [297]:
userSubsetRecs = model.recommendForUserSubset(users, 5)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[6818, 4.9396324...|
|   463|[[7842, 5.1208186...|
|   148|[[89904, 5.070490...|
+------+--------------------+



In [298]:
userSubsetRecs.select("userId","recommendations.movieId").show(5, False)

+------+----------------------------------+
|userId|movieId                           |
+------+----------------------------------+
|471   |[6818, 89904, 6460, 7096, 213]    |
|463   |[7842, 59018, 7841, 60943, 6818]  |
|148   |[89904, 86320, 98491, 2843, 49347]|
+------+----------------------------------+



In [299]:
movies = ratings_df.select("movieId").distinct().limit(3)
movies.show()

+-------+
|movieId|
+-------+
|   1580|
|   2366|
|   3175|
+-------+



In [300]:
movieSubsetRecs = model.recommendForItemSubset(movies, 5)
movieSubsetRecs.select("movieId", "recommendations.UserId").show(5, False)

+-------+----------------------+
|movieId|UserId                |
+-------+----------------------+
|1580   |[53, 12, 543, 452, 43]|
|3175   |[53, 543, 236, 43, 12]|
|2366   |[53, 224, 69, 451, 43]|
+-------+----------------------+



In [301]:
# prediction for some movie for a specific user
movie_ids = [1580,3175,2366,1590]
user_ids = [543,543,543,543]
new_user_preds = sqlContext.createDataFrame(zip(movie_ids, user_ids), schema=['movieId','userId'])

In [302]:
new_predictions = model.transform(new_user_preds)
new_predictions.show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|   1580|   543| 4.6211176|
|   2366|   543| 3.2962856|
|   3175|   543|  4.654319|
|   1590|   543|  2.878671|
+-------+------+----------+



## New combined list

In [303]:
df = ratings_df.join(movies_df, ["movieID"], "left")
df = df.join(tags_df, ["userID", "movieID"], "left")
df.show()

+------+-------+------+---------+--------------------+--------------------+----+---------+
|userId|movieId|rating|timestamp|               title|              genres| tag|timestamp|
+------+-------+------+---------+--------------------+--------------------+----+---------+
|     1|      1|   4.0|964982703|    Toy Story (1995)|Adventure|Animati...|null|     null|
|     1|      3|   4.0|964981247|Grumpier Old Men ...|      Comedy|Romance|null|     null|
|     1|      6|   4.0|964982224|         Heat (1995)|Action|Crime|Thri...|null|     null|
|     1|     47|   5.0|964983815|Seven (a.k.a. Se7...|    Mystery|Thriller|null|     null|
|     1|     50|   5.0|964982931|Usual Suspects, T...|Crime|Mystery|Thr...|null|     null|
|     1|     70|   3.0|964982400|From Dusk Till Da...|Action|Comedy|Hor...|null|     null|
|     1|    101|   5.0|964980868|Bottle Rocket (1996)|Adventure|Comedy|...|null|     null|
|     1|    110|   4.0|964982176|   Braveheart (1995)|    Action|Drama|War|null|     null|

In [304]:
print("Num of users: " + str(df.groupBy("userID").count().count()))
print("Num of movies: " + str(df.groupBy("movieID").count().count()))
print("Num of tags: " + str(df.groupBy("tag").count().count()))
print("Num of rows in ratings: " + str(ratings.count()))
print("Num of rows in tags: " + str(tags.count()))
print("Num of rows in combined dataframe: " + str(df.count()))

Num of users: 610
Num of movies: 9724
Num of tags: 1544
Num of rows in ratings: userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64
Num of rows in tags: userId       3683
movieId      3683
tag          3683
timestamp    3683
dtype: int64
Num of rows in combined dataframe: 102677


In [305]:
# Split training and testing data
# ALS requires numeric data, our data contains strings
training_df, validation_df = df.randomSplit([.8, .2])

In [306]:
# Build another model with combined dataframe
als = ALS(alpha=0, maxIter=iterations, regParam=regularization_parameter, rank=50, userCol="userId", itemCol="movieId", ratingCol="rating")
new_model = als.fit(training_df)
predictions = model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)
print("Root-mean_square error = " + str(rmse))

Root-mean_square error = 0.6548761138109392


In [238]:
predictions.show(n=5)

+------+-------+------+---------+--------------------+------+----+---------+----------+
|userId|movieId|rating|timestamp|               title|genres| tag|timestamp|prediction|
+------+-------+------+---------+--------------------+------+----+---------+----------+
|   597|    471|   2.0|941558175|Hudsucker Proxy, ...|Comedy|null|     null| 3.8203557|
|   436|    471|   3.0|833530187|Hudsucker Proxy, ...|Comedy|null|     null|  3.631647|
|   409|    471|   3.0|967912821|Hudsucker Proxy, ...|Comedy|null|     null| 4.0115466|
|   372|    471|   3.0|874415126|Hudsucker Proxy, ...|Comedy|null|     null| 3.2891638|
|    32|    471|   3.0|856737165|Hudsucker Proxy, ...|Comedy|null|     null| 3.9300745|
+------+-------+------+---------+--------------------+------+----+---------+----------+
only showing top 5 rows



In [309]:
# Make 5 movie recommendations per user using their highest predicted ratings
recommendations = new_model.recommendForAllUsers(5)
# Print Some Prediction
recommendations.registerTempTable("ALS_recs_temp")
clean_recs = spark.sql("SELECT userId, movieIds_and_ratings.movieId AS movieId, movieIds_and_ratings.rating AS prediction FROM ALS_recs_temp LATERAL VIEW explode(recommendations) exploded_table AS movieIds_and_ratings")
clean_recs = clean_recs.join(movies_df, on=['movieId'], how='left').sort('userId')
clean_recs.show()

# join predictions with the movies df to show the movie name
predictions.join(df, "movieId").select("userId","title","genres","prediction").show(5)

+-------+------+----------+--------------------+--------------------+
|movieId|userId|prediction|               title|              genres|
+-------+------+----------+--------------------+--------------------+
|   1262|     1| 5.3404207|Great Escape, The...|Action|Adventure|...|
|   3494|     1| 5.2827806|    True Grit (1969)|Adventure|Drama|W...|
|   5915|     1|  5.210826|Victory (a.k.a. E...|    Action|Drama|War|
|   1204|     1|  5.192298|Lawrence of Arabi...| Adventure|Drama|War|
|   3653|     1| 5.2183743|Endless Summer, T...|         Documentary|
| 106782|     2| 4.6645274|Wolf of Wall Stre...|  Comedy|Crime|Drama|
|  60756|     2| 4.6924524|Step Brothers (2008)|              Comedy|
|  89774|     2| 4.6892004|      Warrior (2011)|               Drama|
|  80906|     2|  4.562723|   Inside Job (2010)|         Documentary|
| 131724|     2| 4.9094415|The Jinx: The Lif...|         Documentary|
|   5919|     3|  4.852443|      Android (1982)|              Sci-Fi|
|   7991|     3|  4.

AnalysisException: Reference 'userId' is ambiguous, could be: userId, userId.;

In [ ]:
# for_one_user = predictions.filter(col("userId")==599).join(movies_df, "movieId").join(links_df, "movieId").select("userId","title","genres","tmdbId","prediction")
# for_one_user.show(5)

In [ ]:
# create code to open web browsers for recommended movies
# import webbrowser
# link = "https://www.themoviedb.org/movie/"
# for movie in for_one_user.take(1):
#     movieURL = link+str(movie.tmdbId)
#     print(movie.title)
#     webbrowser.open(movieURL)

In [180]:
# generate top 5 movie recommendations for each user
userRecomments = new_model.recommendForAllUsers(5)
# generate top 5 user recommendations for each movie
movieRecomments = new_model.recommendForAllItems(5)

In [181]:
userRecomments.select("userId","recommendations.movieId").show(5, False)

+------+------------------------------------+
|userId|movieId                             |
+------+------------------------------------+
|471   |[2324, 89904, 1203, 98491, 79702]   |
|463   |[78836, 5466, 5952, 30745, 159817]  |
|496   |[858, 5618, 7099, 7767, 89759]      |
|148   |[98491, 33649, 160718, 89904, 78836]|
|540   |[78836, 159817, 296, 92535, 177593] |
+------+------------------------------------+
only showing top 5 rows



In [5]:
#df.drop("Unnamed: 0", axis=1, inplace=True)

In [50]:
#df.head()

In [219]:
data = spark.read.csv('../../data/cleaned_movies_df.csv', inferSchema=True, header=True)
data.show()

+---+------+------+-------+--------------------+--------------------+------+---------+
|_c0|userID|rating|movieID|               title|              genres|   tag|timestamp|
+---+------+------+-------+--------------------+--------------------+------+---------+
|  0|     1|   4.0|   1208|Apocalypse Now (1...|    Action|Drama|War|Uknown|     null|
|  1|     1|   4.0|   1348|Nosferatu (Nosfer...|              Horror|Uknown|     null|
|  2|     4|   5.0|    457|Fugitive, The (1993)|            Thriller|Uknown|     null|
|  3|     4|   2.0|    599|Wild Bunch, The (...|   Adventure|Western|Uknown|     null|
|  4|     6|   4.0|    274|Man of the House ...|              Comedy|Uknown|     null|
|  5|     6|   1.0|    327|    Tank Girl (1995)|Action|Comedy|Sci-Fi|Uknown|     null|
|  6|     6|   3.0|    520|Robin Hood: Men i...|              Comedy|Uknown|     null|
|  7|     7|   4.0|   1101|      Top Gun (1986)|      Action|Romance|Uknown|     null|
|  8|     8|   3.0|    235|      Ed Wood (1

In [224]:
from pyspark.sql.functions import col

data = data.where(col("tag").isNotNull())

In [222]:
#data = data.na.drop(subset=["tag"])

In [225]:
data.show()

+---+------+------+-------+--------------------+--------------------+------+---------+
|_c0|userID|rating|movieID|               title|              genres|   tag|timestamp|
+---+------+------+-------+--------------------+--------------------+------+---------+
|  0|     1|   4.0|   1208|Apocalypse Now (1...|    Action|Drama|War|Uknown|     null|
|  1|     1|   4.0|   1348|Nosferatu (Nosfer...|              Horror|Uknown|     null|
|  2|     4|   5.0|    457|Fugitive, The (1993)|            Thriller|Uknown|     null|
|  3|     4|   2.0|    599|Wild Bunch, The (...|   Adventure|Western|Uknown|     null|
|  4|     6|   4.0|    274|Man of the House ...|              Comedy|Uknown|     null|
|  5|     6|   1.0|    327|    Tank Girl (1995)|Action|Comedy|Sci-Fi|Uknown|     null|
|  6|     6|   3.0|    520|Robin Hood: Men i...|              Comedy|Uknown|     null|
|  7|     7|   4.0|   1101|      Top Gun (1986)|      Action|Romance|Uknown|     null|
|  8|     8|   3.0|    235|      Ed Wood (1

In [228]:
import pyspark.sql.functions as f
my_list = data.select(f.collect_list('tag')).first()[0]
my_list

['Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'anti-intellectual',
 'bad writing',
 'dystopia',
 'new society',
 'plot holes',
 'predictable',
 'scifi',
 'unintelligent',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Uknown',
 'Ukn

In [229]:
# Split training and testing data
# ALS requires numeric data, our data contains strings
training_df, validation_df = data.randomSplit([.8, .2])

In [214]:
# # ALS requires the following parameters
iterations = 10
regularization_parameter = 0.1
rank = 4
errors = []
err = 0

In [215]:
# Build another model with combined dataframe
# Build the model
als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=4, userCol="userID", itemCol="movieID", ratingCol="rating")
model = als.fit(training_df)
predictions = model.transform(validation_df)
new_predictions = predictions.filter(col('prediction') != np.nan)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)
print("Root-mean_square error = " + str(rmse))

Py4JJavaError: An error occurred while calling o2999.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 3515.0 failed 1 times, most recent failure: Lost task 1.0 in stage 3515.0 (TID 16584, jeffreys-air, executor driver): java.lang.NullPointerException: Value at index 2 is null
	at org.apache.spark.sql.Row.getAnyValAs(Row.scala:523)
	at org.apache.spark.sql.Row.getFloat(Row.scala:262)
	at org.apache.spark.sql.Row.getFloat$(Row.scala:262)
	at org.apache.spark.sql.catalyst.expressions.GenericRow.getFloat(rows.scala:166)
	at org.apache.spark.ml.recommendation.ALS.$anonfun$fit$2(ALS.scala:700)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:459)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$ConcatIterator.hasNext(Iterator.scala:222)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWr

In [310]:
! pwd

/Users/jeffreyaccomando/Documents/Flatiron/Phase_4_Project/Movie-Recommender/notebooks/exploratory
